In [101]:
import os
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import pandas as py
import json
import traceback
from dotenv import main
import PyPDF2
import traceback


In [102]:
main.load_dotenv()
key = os.getenv('OPEN_AI_KEY')

In [51]:
llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=key, temperature=0.7)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115ed2c80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115ed0220>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [64]:
TEMPLATE = """
Text: {text}
you are an expert MCQ `generator, it your job to create {number} of questions and your subject is {subject}, {tone} will be 
number to between 1 to 10 o define the hardness of the quiz, make there is no repeatred question \n
### response 
{response_json}
"""

In [70]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [71]:
with open("/Users/kartiksharma/DATA/AI DS ML/MCQ generator/Response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [72]:
quiz_gen_promt_template = PromptTemplate(
    input_variables = ['text', 'number', 'subject', 'tone', 'response_json'],
    template = TEMPLATE
)

In [80]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [81]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_gen_promt_template, output_key="quiz", verbose=True)

In [82]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [84]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [85]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText: {text}\nyou are an expert MCQ `generator, it your job to create {number} of questions and your subject is {subject}, {tone} will be \nnumber to between 1 to 10 o define the hardness of the quiz, make there is no repeatred question \n\n### response \n{response_json}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115ed2c80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115ed0220>, openai_api_key=SecretStr('**********'), openai_proxy=''), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only us

In [88]:
PATH = '/Users/kartiksharma/DATA/AI DS ML/MCQ generator/data.txt'

In [93]:
with open(PATH, "r") as file:
    TEXT = file.read()


In [99]:
TEXT
NUMBER = 5
SUBJECT = 'CONTITUION OF INDIA'
TONE = 8
RESPONSE_JSON = RESPONSE_JSON

In [100]:
with get_openai_callback() as cb:
    generate_evaluate_chain(
         {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

/Users/kartiksharma/DATA/AI DS ML/MCQ generator/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: PART III
FUNDAMENTAL RIGHTS
General
12. Definition.—In this Part, unless the context otherwise requires, “the
State” includes the Government and Parliament of India and the Government
and the Legislature of each of the States and all local or other authorities wit
hin
the territory of India or under the control of the Government of India.
13. Laws inconsistent with or in derogation of the fundamental
rights.—(1) All laws in force in the territory of India immediately before the
commencement of this Constitution, in so far as they are inconsistent with the
provisions of this Part, shall, to the extent of such inconsistency, be void.
(2) The State shall not make any law which takes away or abridges the
rights conferred by this Part and any law made in contravention of this clause
shall, to the extent of the contravention, be void.
(3) In this article, unless the context otherwise 

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [107]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%S')

'07_31_2024_17_51'